In [1]:
import pandas as pd
import numpy as np
import os
import gc
import random
import time

import json
from tqdm import tqdm
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, average_precision_score


In [2]:
# read tagged file
IOB_PATH = "C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\"
#IOB_files = [f for f in os.listdir(IOB_PATH) if os.path.getctime(IOB_PATH+f) > 1694707200] # from 9/15
IOB_files = [f for f in os.listdir(IOB_PATH)]
IOB_files

['1820896.txt',
 '1821329.txt',
 '1823904.txt',
 '1825249.txt',
 '1892579.txt',
 '1892580.txt',
 '1892581.txt',
 '1892584.txt',
 '1892804.txt',
 '1892889.txt',
 '1892890.txt',
 '1894040.txt',
 '1894041.txt',
 '1895455.txt',
 '1895992.txt',
 '1896495.txt',
 '1899461.txt',
 '1899597.txt',
 '1899599.txt',
 '2120297.txt',
 '2120565.txt',
 '2122206.txt',
 '2124241.txt',
 '2124242.txt',
 '2124243.txt',
 '2124244.txt',
 '2124245.txt',
 '2124246.txt',
 '2124247.txt',
 '2124248.txt',
 '2124249.txt',
 '2124250.txt',
 '2124252.txt',
 '2124257.txt',
 '2124259.txt',
 '2124538.txt',
 '2124819.txt',
 '2124824.txt',
 '2124826.txt',
 '2124827.txt',
 '2124828.txt',
 '2124829.txt',
 '2124830.txt',
 '2124831.txt',
 '2124832.txt',
 '2124835.txt',
 '2124836.txt',
 '2125108.txt',
 '2125315.txt',
 '2129098.txt',
 '2129822.txt',
 '2148521.txt',
 '2148878.txt',
 '2168383.txt',
 '2168424.txt',
 '2169058.txt',
 '2169059.txt',
 '2169060.txt',
 '2169591.txt',
 '2169598.txt',
 '2169599.txt',
 '2169853.txt',
 '216985

In [3]:
# read untagged file
unIOB_PATH = "C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\unIOB\\"
unIOB_files = [f for f in os.listdir(unIOB_PATH)] #since 7/28
unIOB_files

['1820896.txt',
 '1821329.txt',
 '1823904.txt',
 '1825249.txt',
 '1892579.txt',
 '1892580.txt',
 '1892581.txt',
 '1892584.txt',
 '1892804.txt',
 '1892885.txt',
 '1892889.txt',
 '1892890.txt',
 '1893751.txt',
 '1894040.txt',
 '1894041.txt',
 '1895455.txt',
 '1895992.txt',
 '1896495.txt',
 '1899461.txt',
 '1899597.txt',
 '1899599.txt',
 '2120297.txt',
 '2120565.txt',
 '2122206.txt',
 '2124241.txt',
 '2124242.txt',
 '2124243.txt',
 '2124244.txt',
 '2124245.txt',
 '2124246.txt',
 '2124247.txt',
 '2124248.txt',
 '2124249.txt',
 '2124250.txt',
 '2124252.txt',
 '2124257.txt',
 '2124258.txt',
 '2124259.txt',
 '2124263.txt',
 '2124536.txt',
 '2124537.txt',
 '2124538.txt',
 '2124819.txt',
 '2124824.txt',
 '2124826.txt',
 '2124827.txt',
 '2124828.txt',
 '2124829.txt',
 '2124830.txt',
 '2124831.txt',
 '2124832.txt',
 '2124835.txt',
 '2124836.txt',
 '2125108.txt',
 '2125315.txt',
 '2125384.txt',
 '2129098.txt',
 '2129822.txt',
 '2147325.txt',
 '2148521.txt',
 '2148878.txt',
 '2168171.txt',
 '216819

In [15]:
for f in IOB_files:
    fd1 = open(IOB_PATH+f, mode='r', encoding='utf-8')
    tokens1 = []
    try: 
        tokens1 = fd1.read().split('\n')
    except UnicodeDecodeError:
        fd1.close()
        print(f"decoding big5 error: {f}")
        try :
            fd1 = open(IOB_PATH+f, mode='r', encoding='ansi')
            tokens1 = fd1.read().split('\n')
        except UnicodeDecodeError:
            fd1.close()
            print(f"decoding ansi error: {f}")
            continue
    fd2 = open(unIOB_PATH+f, mode='r', encoding='utf-8')
    tokens2 = fd2.read().split('\n')
    fd1.close()
    fd2.close()
    print(f)
    bias = 0
    fd = open(IOB_PATH+f, mode='w', encoding='utf-8')
    for i in range(len(tokens1)):
        token1 = tokens1[i].split('\t')
        if token1[0] == '':
            continue
        token2 = tokens2[i+bias].split('\t')
        answer = ''
        
        if len(token1) == 3:
            print(f"{f}: redundant pos")
            token1[0] = token1[1]
            token1[1] = token1[2]
            print(f"new token: {token1}")
        
        if len(token1) == 1 and token1[0] != '':
            print(f"{f}: missing category in {i} lines")
            answer = input(f"y for fixing this issue or n for ignoring it.")
            if answer == 'y':
                token1.append('O')
                print(f"new token: {token1}")
        if token1[0] != token2[0]:
            print(f"{f}: may have encoding issue in {i} lines")
            if (token1[0] == '","' or token1[0] == '""","""') and token2[0] == ',':
                print(f"{token1[0]} -> {token2[0]}")
                answer = 'y'
            else: 
                answer = input(f"y for replacing new word or n for ignoring it: {token1[0]} -> {token2[0]}")
            if answer == 'y':
                token1[0] = token2[0]
                print(f"new token: {token1}")
            elif answer == 'a':
                fd.write(token2[0]+'\t'+'O'+'\n')
                bias = bias+1
            elif answer == 'n':
                bias = bias+1
            else:
                bias = bias-1
                
        if token1[0] == '':
            print(f"{f}: empty word in {i} lines")
            if i == len(tokens1)-1:
                answer = 'y'
            else:
                answer = input(f"y for deleting this word or n for ignoring it.")
            if answer == 'y':
                continue
            elif answer == 'n':
                fd.write(token1[0]+'\t'+token1[1])
        elif token1[0] == '""","""' or token1[0] == '","':
            print(f"{f}: \",\" issue in {i} lines")
            answer = input(f"y for fixing this word or n for ignoring it.")
            if answer == 'y':
                fd.write(','+'\t'+token1[1])
            elif answer == 'n':
                fd.write(token1[0]+'\t'+token1[1])
        else:
            fd.write(token1[0]+'\t'+token1[1])
        
        if i != len(tokens1)-1 and tokens1[i+1].split('\t')[0] != '':
            fd.write('\n')
    
    fd.close()
        
        

3090315.txt
3090316.txt
3090317.txt
3090318.txt
3090319.txt
3090320.txt
3090321.txt
3090322.txt
3090323.txt
3090324.txt
3090325.txt
3090326.txt
3090327.txt
3090328.txt
3090331.txt
3090331.txt: may have encoding issue in 158 lines
y for replacing new word or n for ignoring it: made.ps1 -> made.ps1′n
3090331.txt: may have encoding issue in 159 lines
y for replacing new word or n for ignoring it: ′ -> ;n
3090331.txt: may have encoding issue in 160 lines
y for replacing new word or n for ignoring it: ) -> 2s
3090331.txt: may have encoding issue in 161 lines
y for replacing new word or n for ignoring it: ; -> 2s
3090331.txt: may have encoding issue in 162 lines
y for replacing new word or n for ignoring it: made.ps1 -> 2s
3090331.txt: may have encoding issue in 163 lines
y for replacing new word or n for ignoring it: ″ -> 2n
3090331.txt: may have encoding issue in 164 lines
y for replacing new word or n for ignoring it: 2 -> &s
3090331.txt: may have encoding issue in 165 lines
y for replaci

In [12]:
fd.close()

In [17]:
for f in IOB_files:
    i = 0
    fd = open(IOB_PATH+f, mode='r', encoding='utf8')
    tokens = []
    try:
        tokens = fd.read().split('\n')
    except UnicodeDecodeError:
        print(f)
        fd.close()
        fd = open(IOB_PATH+f, mode='r', encoding='big5')
        txt = fd.read()
        fd.close()
        fd = open(IOB_PATH+f, mode='w', encoding='utf8')
        fd.write(txt)
    fd.close()
    #print(tokens)
    B_Flag = False
    while i < len(tokens):
        tag = ""
        try:
            tag = (tokens[i].split('\t'))[1]
        except :
            print(f)
            exit()
        if "B-" in tag:
            B_Flag = True
        elif B_Flag and "O" in tag:
            B_Flag = False
        elif B_Flag == False and "I-" in tag:
            print(f"{f}: tagging error in {i} lines")
        elif B_Flag == True and "I-" in tag:
            i+=1
            continue
        elif tag == 'O':
            i+=1
            continue
        else:
            print(f"error tag in {f} in {i} lines")
        i += 1

In [3]:
# Analysis of positive instances in each fold of our CV folds
def Create_metadata(file_list):
    Malware_sum = []
    Vulnerability_sum = []
    Application_sum = []
    Indicator_sum = []
    TTPs_sum = []
    ThreatActor_sum = []
    Reporter_sum = []
    Protocol_sum = []
    O_sum = []
    count = []
    group = []

    metadata = pd.DataFrame(file_list,columns=['Id'])

    # Here I am using the metadata file available during training. Since the code will run again during submission, if 
    # I used the usual file from the competition folder, it would have been updated with the test files too.

    for f in tqdm(file_list):
        fpath = IOB_PATH + f
        fd = open(fpath, encoding='utf-8')
        tokens = fd.read().split('\n')

        malware = 0
        vulnerability = 0
        application = 0
        indicator = 0
        ttps = 0
        threatactor = 0
        reporter = 0
        protocol = 0
        o = 0

        for i in range(len(tokens)):

            tag = (tokens[i].split('\t'))[1]
            if tag == "B-Malware":
                malware += 1
            elif tag == "B-Vulnerability":
                vulnerability += 1
            elif tag == "B-Application":
                application += 1
            elif tag == "B-TTPs":
                ttps += 1
            elif tag == "B-Indicator":
                indicator += 1
            elif tag == "B-ThreatActor":
                threatactor += 1
            elif tag == "B-Reporter":
                reporter += 1
            elif tag == "B-Protocol":
                protocol += 1
            elif tag == "O":
                o += 1

        Malware_sum.append(malware)
        Vulnerability_sum.append(vulnerability)
        Application_sum.append(application)
        Indicator_sum.append(indicator)
        TTPs_sum.append(ttps)
        ThreatActor_sum.append(threatactor)
        Reporter_sum.append(reporter)
        Protocol_sum.append(protocol)
        O_sum.append(o)

        count.append(len(tokens))
        tmp = int( (f.split('.txt')[0]) )/10000
        group.append(int(tmp))

    print(f"{len(file_list)} files have positive values in all 8 classes")

    metadata['Malware_sum'] = Malware_sum
    metadata['Vulnerability_sum'] = Vulnerability_sum
    metadata['Application_sum'] = Application_sum
    metadata['Indicator_sum'] = Indicator_sum
    metadata['TTPs_sum'] = TTPs_sum
    metadata['ThreatActor_sum'] = ThreatActor_sum
    metadata['Reporter_sum'] = Reporter_sum
    metadata['Protocol_sum'] = Protocol_sum
    metadata['O_sum'] = O_sum
    metadata['count'] = count
    metadata['group'] = group
    print(metadata)
    return metadata


In [5]:
metadata = Create_metadata(IOB_files)
tmp_x = metadata[['Malware_sum', 'Vulnerability_sum', 'Application_sum', 'Indicator_sum', 'TTPs_sum', 'ThreatActor_sum', 'Reporter_sum', 'Protocol_sum']].values
sgkf = StratifiedShuffleSplit(n_splits=10, train_size=0.7, test_size=0.3, random_state=42)
for i, (train_index, test_index) in enumerate(sgkf.split(X=tmp_x,y=[1]*len(metadata),groups=None)):
    print(f"Fold = {i}")
    train_ids = metadata.loc[train_index, 'Id']
    test_ids = metadata.loc[test_index, 'Id']
    
    print(f"Length of Train = {len(train_index)}, Length of Test = {len(test_index)}")
    Malware_sum = metadata.loc[train_index, 'Malware_sum'].sum()
    Vulnerability_sum = metadata.loc[train_index, 'Vulnerability_sum'].sum()
    Application_sum = metadata.loc[train_index, 'Application_sum'].sum()
    Indicator_sum = metadata.loc[train_index, 'Indicator_sum'].sum()
    TTPs_sum = metadata.loc[train_index, 'TTPs_sum'].sum()
    ThreatActor_sum = metadata.loc[train_index, 'ThreatActor_sum'].sum()
    Reporter_sum = metadata.loc[train_index, 'Reporter_sum'].sum()
    Protocol_sum = metadata.loc[train_index, 'Protocol_sum'].sum()
    O_sum = metadata.loc[train_index, 'O_sum'].sum()
    print(f"Train classes: {Malware_sum:,}, {Vulnerability_sum:,}, {Application_sum:,}, {Indicator_sum:,}, {TTPs_sum:,}, {ThreatActor_sum:,}, {Reporter_sum:,}, {Protocol_sum:,}, {O_sum:,}")
    
    Malware_sum = metadata.loc[test_index, 'Malware_sum'].sum()
    Vulnerability_sum = metadata.loc[test_index, 'Vulnerability_sum'].sum()
    Application_sum = metadata.loc[test_index, 'Application_sum'].sum()
    Indicator_sum = metadata.loc[test_index, 'Indicator_sum'].sum()
    TTPs_sum = metadata.loc[test_index, 'TTPs_sum'].sum()
    ThreatActor_sum = metadata.loc[test_index, 'ThreatActor_sum'].sum()
    Reporter_sum = metadata.loc[test_index, 'Reporter_sum'].sum()
    Protocol_sum = metadata.loc[test_index, 'Protocol_sum'].sum()
    O_sum = metadata.loc[test_index, 'O_sum'].sum()
    print(f"Test classes: {Malware_sum:,}, {Vulnerability_sum:,}, {Application_sum:,}, {Indicator_sum:,}, {TTPs_sum:,}, {ThreatActor_sum:,}, {Reporter_sum:,}, {Protocol_sum:,}, {O_sum:,}")
    
# FOLD 1 is the most well balanced
# The actual train-test split (based on Fold 1)


100%|██████████████████████████████████████████████████████████████████████████████| 692/692 [00:00<00:00, 6643.12it/s]

692 files have positive values in all 8 classes
               Id  Malware_sum  Vulnerability_sum  Application_sum  \
0     1820896.txt            0                  0                0   
1     1821329.txt            0                  0                0   
2     1823904.txt            0                  0                0   
3     1825249.txt            0                  0                0   
4     1892579.txt            1                  0                0   
..            ...          ...                ...              ...   
687  44243065.txt            1                  0                0   
688  44243070.txt            1                  0                0   
689  44243097.txt            1                  0                0   
690  44243245.txt            1                  0                0   
691  44243250.txt            1                  0                0   

     Indicator_sum  TTPs_sum  ThreatActor_sum  Reporter_sum  Protocol_sum  \
0                0         0      

In [6]:
tmp_x = metadata[['Malware_sum', 'Vulnerability_sum', 'Application_sum', 'Indicator_sum', 'TTPs_sum', 'ThreatActor_sum', 'Reporter_sum', 'Protocol_sum']].values
sgkf = StratifiedShuffleSplit(n_splits=10, train_size=0.7, test_size=0.3, random_state=42)
for i, (train_index, test_index) in enumerate(sgkf.split(X=tmp_x,y=[1]*len(metadata),groups=None)):
    if i != 3:
        continue
    print(f"Fold = {i}")
    train_ids = metadata.loc[train_index, 'Id']
    test_ids = metadata.loc[test_index, 'Id']
    print(f"Length of Train = {len(train_ids)}, Length of Test = {len(test_ids)}")
    
    if i == 3:
        break
        
train_fpaths = [IOB_PATH + _id for _id in train_ids]

Fold = 3
Length of Train = 484, Length of Test = 208


In [8]:
metadata = Create_metadata(test_ids.values)
tmp_x = metadata[['Malware_sum', 'Vulnerability_sum', 'Application_sum', 'Indicator_sum', 'TTPs_sum', 'ThreatActor_sum', 'Reporter_sum', 'Protocol_sum']].values
sgkf = StratifiedShuffleSplit(n_splits=10, train_size=0.5, test_size=0.5, random_state=40)
for i, (train_index, valid_index) in enumerate(sgkf.split(X=tmp_x,y=[1]*len(metadata),groups=None)):
    print(f"Fold = {i}")
    train_ids2 = metadata.loc[train_index, 'Id']
    valid_ids = metadata.loc[valid_index, 'Id']
    
    print(f"Length of Train = {len(train_index)}, Length of Valid = {len(valid_index)}")
    Malware_sum = metadata.loc[train_index, 'Malware_sum'].sum()
    Vulnerability_sum = metadata.loc[train_index, 'Vulnerability_sum'].sum()
    Application_sum = metadata.loc[train_index, 'Application_sum'].sum()
    Indicator_sum = metadata.loc[train_index, 'Indicator_sum'].sum()
    TTPs_sum = metadata.loc[train_index, 'TTPs_sum'].sum()
    ThreatActor_sum = metadata.loc[train_index, 'ThreatActor_sum'].sum()
    Reporter_sum = metadata.loc[train_index, 'Reporter_sum'].sum()
    Protocol_sum = metadata.loc[train_index, 'Protocol_sum'].sum()
    O_sum = metadata.loc[train_index, 'O_sum'].sum()
    count = metadata.loc[train_index, 'count'].sum()
    print(f"Train classes: {Malware_sum:,}, {Vulnerability_sum:,}, {Application_sum:,}, {Indicator_sum:,}, {TTPs_sum:,}, {ThreatActor_sum:,}, {Reporter_sum:,}, {Protocol_sum:,}, {O_sum:,}")
    
    Malware_sum = metadata.loc[valid_index, 'Malware_sum'].sum()
    Vulnerability_sum = metadata.loc[valid_index, 'Vulnerability_sum'].sum()
    Application_sum = metadata.loc[valid_index, 'Application_sum'].sum()
    Indicator_sum = metadata.loc[valid_index, 'Indicator_sum'].sum()
    TTPs_sum = metadata.loc[valid_index, 'TTPs_sum'].sum()
    ThreatActor_sum = metadata.loc[valid_index, 'ThreatActor_sum'].sum()
    Reporter_sum = metadata.loc[valid_index, 'Reporter_sum'].sum()
    Protocol_sum = metadata.loc[valid_index, 'Protocol_sum'].sum()
    O_sum = metadata.loc[valid_index, 'O_sum'].sum()
    count = metadata.loc[valid_index, 'count'].sum()
    print(f"Valid classes: {Malware_sum:,}, {Vulnerability_sum:,}, {Application_sum:,}, {Indicator_sum:,}, {TTPs_sum:,}, {ThreatActor_sum:,}, {Reporter_sum:,}, {Protocol_sum:,}, {O_sum:,}")
    
# FOLD 1 is the most well balanced
# The actual train-test split (based on Fold 1)


100%|██████████████████████████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 4604.21it/s]

208 files have positive values in all 8 classes
               Id  Malware_sum  Vulnerability_sum  Application_sum  \
0    26736431.txt            0                  0                0   
1    24225346.txt            0                  0                0   
2     2690530.txt            0                  0                0   
3    26125876.txt            1                  0                1   
4     2120565.txt            0                  0                0   
..            ...          ...                ...              ...   
203   2568984.txt            0                  0                0   
204   2925918.txt            0                  0                0   
205  27388653.txt            0                  0                0   
206   2838749.txt            0                  1                1   
207   2567841.txt            0                  2                0   

     Indicator_sum  TTPs_sum  ThreatActor_sum  Reporter_sum  Protocol_sum  \
0                0         2      

In [9]:
tmp_x = metadata[['Malware_sum', 'Vulnerability_sum', 'Application_sum', 'Indicator_sum', 'TTPs_sum', 'ThreatActor_sum', 'Reporter_sum', 'Protocol_sum']].values
sgkf = StratifiedShuffleSplit(n_splits=10, train_size=0.5, test_size=0.5, random_state=40)
for i, (test_index, valid_index) in enumerate(sgkf.split(X=tmp_x,y=[1]*len(metadata),groups=None)):
    if i != 3:
        continue
    print(f"Fold = {i}")
    test_ids2 = metadata.loc[test_index, 'Id']
    valid_ids = metadata.loc[valid_index, 'Id']
    print(f"Length of Train = {len(test_ids2)}, Length of Valid = {len(valid_ids)}")
    
    if i == 3:
        break
        
test_fpaths = [IOB_PATH + _id for _id in test_ids2]
valid_fpaths = [IOB_PATH + _id for _id in valid_ids]

Fold = 3
Length of Train = 104, Length of Valid = 104


In [10]:
#print(train_fpaths)
print(test_fpaths)
#print(valid_fpaths)

['C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\27636498.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\26064401.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2567895.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2524933.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\28719529.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2567836.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2546063.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2524919.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\26123528.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\32041477.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2729473.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\2567897.txt', 'C:\\Users\\chun\\Documents\\Github\\MISP_CTI_dataset\\IOB\\27388629.txt', 'C:\\Users\\chun\\Documents\\Gi

In [24]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [11]:
LBRACKETS = ['(', '[', '{', '<']
RBRACKETS = [')', ']', '}', '>']

In [12]:
def dataset_generation(filepath, filename):
    fd = open(f"C:\\Users\\chun\\Desktop\\MISP\\dataset\\{filename}", mode='w', encoding='utf-8')
    for f in filepath:
        fd1 = open(f, mode='r', encoding='utf-8')
        txt = fd1.read()
        fd1.close()

        option = 2
        bflag = False
        ## option 1: Split each document into an entry
        if option == 1:
            fd.write(txt)
            fd.write('\n\n')

        ## option 2: Split each sentence into an entry
        elif option == 2:
            tokens = txt.split('\n')
            for i in range(len(tokens)):
                fd.write(tokens[i]+'\n')
                if (tokens[i].split('\t'))[0] in LBRACKETS:
                    bflag = True
                elif (tokens[i].split('\t'))[0] in RBRACKETS:
                    bflag = False
                    
                if i == len(tokens)-1:
                    fd.write('\n')
                elif (tokens[i].split('\t'))[0] == '.' and bflag == False:
                    fd.write('\n')

    fd.close()

In [13]:
# Generate train data
dataset_generation(filepath=train_fpaths, filename='train.txt')
# Generate test data
dataset_generation(filepath=test_fpaths, filename='test.txt')
# Generate valid data
dataset_generation(filepath=valid_fpaths, filename='valid.txt')

In [26]:
# Generate test data
fd = open("C:\\Users\\chun\\Desktop\\MISP\\dataset\\test.txt", mode='w', encoding='utf-8')
for f in test_fpaths:
    fd1 = open(f, mode='r', encoding='utf-8')
    txt = fd1.read()
    fd1.close()
    option = 2
    ## option 1: Split each document into an entry
    if option == 1:
        fd.write(txt)
        fd.write('\n\n')
        
    ## option 2: Split each sentence into an entry
    elif option == 2:
        tokens = txt.split('\n')
        for i in range(len(tokens)):
            fd.write(tokens[i]+'\n')
            if (tokens[i].split('\t'))[0] == '.' or i == len(tokens)-1:
                fd.write('\n')
    
fd.close()

In [27]:
# Generate valid data
fd = open("C:\\Users\\chun\\Desktop\\MISP\\dataset\\valid.txt", mode='w', encoding='utf-8')
for f in valid_fpaths:
    fd1 = open(f, mode='r', encoding='utf-8')
    txt = fd1.read()
    fd1.close()
    option = 2
    ## option 1: Split each document into an entry
    if option == 1:
        fd.write(txt)
        fd.write('\n\n')
        
    ## option 2: Split each sentence into an entry
    elif option == 2:
        tokens = txt.split('\n')
        for i in range(len(tokens)):
            fd.write(tokens[i]+'\n')
            if (tokens[i].split('\t'))[0] == '.' or i == len(tokens)-1:
                fd.write('\n')
    
fd.close()

In [32]:
test_list=[]
for f in valid_ids2:
    test_list.append(int((f.split('.txt'))[0]))

test_list


[1820896,
 1892581,
 1896495,
 2124241,
 2124246,
 2124257,
 2124827,
 2124835,
 2129822,
 2168208,
 2169059,
 2169855,
 2184674,
 2191004,
 2208746,
 2281186,
 24225346,
 2464070,
 2524863,
 2524870,
 2524877,
 2524883,
 2524914,
 2524920,
 2524923,
 2524974,
 2527822,
 2548607,
 2549309,
 2552062,
 2552075,
 2555962,
 2558901,
 2567801,
 2567808,
 2567821,
 2567834,
 2567841,
 2567847,
 2567852,
 2567862,
 2567869,
 2567880,
 2567884,
 2567898,
 2567904,
 2567909,
 2567916,
 2567922,
 2567928,
 2569764,
 2570768,
 26055518,
 26055537,
 26055542,
 26076669,
 26121529,
 26123529,
 26256638,
 26281323,
 26577845,
 26737085,
 2677420,
 26805102,
 2682589,
 2688230,
 2696273,
 2698147,
 2709702,
 2710987,
 2729473,
 2737331,
 27388618,
 27388624,
 27388630,
 27388633,
 27388653,
 27388656,
 27559587,
 27598199,
 27636497,
 2838749,
 2848785,
 2852931,
 28855701,
 28855704,
 2961635,
 29755053,
 2988225,
 3002378,
 3003480,
 3004378,
 3011286,
 3017679,
 3017792,
 30746801,
 3075645,
 3077

In [33]:
# read all attributes
attribute_dtype={'id':int, 'type':str, 'category':str, 'to_ids':bool, 'event_id':int, 'timestamp':int, 'comment':str, 'object_id':int, 'object_relation':str, 'value':str}
attributes= pd.read_csv('./Desktop/MISP/MISP_data/Attributes.csv.gz', compression='gzip', index_col='id', dtype=attribute_dtype)
print(attributes.shape)
attributes = attributes[~attributes.duplicated(keep='first')]
print(attributes.shape)

(4916990, 9)
(4906594, 9)


In [34]:
# extracting "text" attributes as MISP CTI
sentences = attributes[pd.DataFrame([attributes['type']=='text', attributes['type']=='comment']).any()]
sentences = sentences[~sentences['value'].duplicated(keep='first')]
sentences

,type,category,to_ids,event_id,timestamp,comment,object_id,object_relation,value
id,,,,,,,,,
47376134,text,Other,False,155523,1680787770,NaN,4024216,scheme,https
47376135,text,Other,False,155523,1680787770,NaN,4024216,resource_path,/hk/ne/porcel.php
47385787,text,Other,False,155523,1680880227,NaN,4025023,scheme,http
47385788,text,Other,False,155523,1680880227,NaN,4025023,resource_path,/ID91863
47414371,text,Other,False,155523,1681198180,NaN,4027577,resource_path,/dhl/
...,...,...,...,...,...,...,...,...,...
59097804,text,Other,False,182280,1687084827,NaN,0,NaN,An issue was discovered in SugarCRM Enterprise...
59097806,text,Other,False,182281,1687084835,NaN,0,NaN,An issue was discovered in SugarCRM Enterprise...
59097808,text,Other,False,182282,1687084837,NaN,0,NaN,An issue was discovered in SugarCRM Enterprise...


In [35]:
fd = open("C:\\Users\\chun\\Desktop\\MISP\\dataset\\plainvalid.txt", mode='w', encoding='utf-8')
for index, value in sentences.iterrows():
    if index in test_list or value['object_id'] in test_list:
        fd.write(value['value']+'\n')
        
fd.close()